In [1]:
import pandas as pd
from collections import Counter
import numpy as np
import glob

# iHART
Get balanced samples for sex-associated coverage.
- 1 neurotypical male + 1 neurotypical female from each family

In [2]:
# Get files actually on AWS 
%%bash
aws s3 ls ihart-hg38/cram/ |  awk '{ print $4 }'  > /scratch/groups/dpwall/personal/briannac/unmapped_reads/data/file_dirs_on_aws.tsv

SyntaxError: invalid syntax (<ipython-input-2-1dffac2cc967>, line 3)

In [9]:
bam_mappings = pd.read_csv('/scratch/groups/dpwall/personal/briannac/unmapped_reads/data/bam_mappings.csv', sep='\t')
bam_mappings.index = bam_mappings.sample_id
aws_files= pd.read_csv('/scratch/groups/dpwall/personal/briannac/unmapped_reads/data/file_dirs_on_aws.tsv', header=None)[0].values
bam_mappings = bam_mappings.loc[list(set(bam_mappings.index).intersection(set([a.replace('.final.cram', '').replace('.crai', '') for a in aws_files])))]


In [3]:
children = bam_mappings[(bam_mappings.relationship=='sibling') & (pd.isna(bam_mappings.derived_affected_status))]
parents = bam_mappings[(bam_mappings.relationship!='sibling') & (pd.isna(bam_mappings.derived_affected_status))]

both_sex_fams = children.groupby('family').aggregate(lambda x: ('1.0' in list(x)) & ('2.0' in list(x)))
both_parents_fams = parents.groupby('family').aggregate(lambda x: ('mother' in list(x)) & ('father' in list(x)))

children = children[[((f in set(both_sex_fams[both_sex_fams.sex_numeric].index))
                     & (f in set(both_parents_fams[both_parents_fams.relationship].index)))  for f in children.family.values]]
males = children[children.sex_numeric=='1.0']
_, idx = np.unique(males.family, return_index=True)
unaff_males = males.iloc[idx]
females = children[children.sex_numeric=='2.0']
_, idx = np.unique(females.family, return_index=True)
unaff_females = females.iloc[idx]


males = children[children.sex_numeric=='1.0']
_, idx = np.unique(males.family, return_index=True)
aff_males = males.iloc[idx]
females = children[children.sex_numeric=='2.0']
_, idx = np.unique(females.family, return_index=True)
aff_females = females.iloc[idx]

In [4]:
children = bam_mappings[(bam_mappings.relationship=='sibling') & (pd.isna(bam_mappings.derived_affected_status))]
parents = bam_mappings[(bam_mappings.relationship!='sibling')]
children = bam_mappings[(bam_mappings.relationship=='sibling') & (pd.isna(bam_mappings.derived_affected_status))]

both_sex_fams = children.groupby('family').aggregate(lambda x: ('1.0' in list(x)) & ('2.0' in list(x)))
same_val_fams = children.groupby('family').aggregate(lambda x: len(set(x))==1)
both_parents_fams = parents.groupby('family').aggregate(lambda x: ('mother' in list(x)) & ('father' in list(x)) & (len(x)==2))

children = children[[(
    (f in set(both_sex_fams[both_sex_fams.sex_numeric].index)) & 
    (f in set(same_val_fams[same_val_fams.race].index)) & 
    (f in set(same_val_fams[same_val_fams.race_ethnic].index)) & 
    (f in set(both_parents_fams[both_parents_fams.relationship].index)) &
    (f in set(same_val_fams[same_val_fams.FatherID].index)) &
    (f in set(same_val_fams[same_val_fams.MotherID].index))) for f in children.family.values]]

males = children[children.sex_numeric=='1.0']
_, idx = np.unique(males.family, return_index=True)
unaff_males = males.iloc[idx].sort_values(['race', 'race_ethnic', 'family'])
females = children[children.sex_numeric=='2.0']
_, idx = np.unique(females.family, return_index=True)
unaff_females = females.iloc[idx].sort_values(['race', 'race_ethnic', 'family'])


children = bam_mappings[(bam_mappings.relationship=='sibling') & (~pd.isna(bam_mappings.derived_affected_status))]

both_sex_fams = children.groupby('family').aggregate(lambda x: ('1.0' in list(x)) & ('2.0' in list(x)))
same_val_fams = children.groupby('family').aggregate(lambda x: len(set(x))==1)
both_parents_fams = parents.groupby('family').aggregate(lambda x: ('mother' in list(x)) & ('father' in list(x)) & (len(x)==2))

children = children[[(
    (f in set(both_sex_fams[both_sex_fams.sex_numeric].index)) & 
    (f in set(same_val_fams[same_val_fams.race].index)) & 
    (f in set(same_val_fams[same_val_fams.derived_affected_status].index)) & 
    (f in set(same_val_fams[same_val_fams.race_ethnic].index)) & 
    (f in set(both_parents_fams[both_parents_fams.relationship].index)) &
    (f in set(same_val_fams[same_val_fams.FatherID].index)) &
    (f in set(same_val_fams[same_val_fams.MotherID].index))) for f in children.family.values]]

males = children[children.sex_numeric=='1.0']
_, idx = np.unique(males.family, return_index=True)
aff_males = males.iloc[idx].sort_values(['race', 'race_ethnic', 'family'])
females = children[children.sex_numeric=='2.0']
_, idx = np.unique(females.family, return_index=True)
aff_females = females.iloc[idx].sort_values(['race', 'race_ethnic', 'family'])


In [17]:
len(aff_females)

212

In [5]:
selected_children = pd.concat([unaff_males, unaff_females, aff_males.iloc[:100], aff_females[:100]])
selected_parents = parents[[f in set(selected_children.family) for f in parents.family]]

In [29]:
## Check if all CRAMS are copied.

for iteration, i in enumerate(selected_children.index):
    if not len(glob.glob('/scratch/groups/dpwall/personal/briannac/unmapped_reads/pileups/data/crams/' + i + '.cram')):
        print(i, iteration+1)
        
for iteration, i in enumerate(selected_parents.index):
    if not len(glob.glob('/scratch/groups/dpwall/personal/briannac/unmapped_reads/pileups/data/crams/' + i + '.cram')):
        print(i, iteration+1)
print("ALL crams copied!")

ALL crams copied!


In [7]:
selected_children['file_loc'] = ['s3://ihart-hg38/cram/%s.final.cram' %i for i in selected_children.index]
selected_parents['file_loc'] = ['s3://ihart-hg38/cram/%s.final.cram' %i for i in selected_parents.index]

/home/users/briannac/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
selected_children['batch'] = ['batch_' + b.split('-')[-1] for b in selected_children.sequencing_plate.values]
selected_parents['batch'] = ['batch_' + b.split('-')[-1] for b in selected_parents.sequencing_plate.values]


/home/users/briannac/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
selected_children[['file_loc', 'batch']].to_csv('/scratch/groups/dpwall/personal/briannac/unmapped_reads/data/balanced_children_ihart.csv',
                                   header=None, sep='\t')
selected_parents[['file_loc', 'batch']].to_csv('/scratch/groups/dpwall/personal/briannac/unmapped_reads/data/balanced_parents_ihart.csv',
                                   header=None, sep='\t')

In [23]:
### parents + children dataframe (sorted)

In [24]:
children_df = pd.read_csv('/scratch/groups/dpwall/personal/briannac/unmapped_reads/data/balanced_children_ihart.csv', sep='\t', header=None)
parents_df = pd.read_csv('/scratch/groups/dpwall/personal/briannac/unmapped_reads/data/balanced_parents_ihart.csv', sep='\t', header=None)

In [25]:
children_and_parents_df = pd.concat([children_df, parents_df])
children_and_parents_df.index = children_and_parents_df[0]

In [26]:
children_and_parents_df = children_and_parents_df.loc[
    list(bam_mappings.loc[children_and_parents_df[0]].sort_values('family').index)].drop([0], axis=1)
children_and_parents_df.to_csv(
    '/scratch/groups/dpwall/personal/briannac/unmapped_reads/data/parents_and_children.csv', header=None, sep='\t')

# 10000 Genomes

In [ ]:
## Print ids for copy paste in bash script.
idx = [g.split('/')[-1].split('.final')[0] for g in glob.glob(
    '/scratch/groups/dpwall/personal/chloehe/unmapped_reads/bam/batch_00514/*.final.cram')]
' '.join(idx)

In [15]:
kg_ped = pd.read_csv('/scratch/groups/dpwall/personal/briannac/unmapped_reads/data/1kg_related_samples.ped', sep=' ')


In [28]:
Counter([(i,j) for i,j in zip(kg_ped.FatherID.values, kg_ped.MotherID.values)]).most_common()

[(('0', '0'), 2590),
 (('HG00656', 'HG00657'), 2),
 (('HG03943', 'HG03944'), 2),
 (('HG03679', 'HG03642'), 2),
 (('NA19661', 'NA19660'), 2),
 (('NA19679', 'NA19678'), 2),
 (('0', 'HG00144'), 1),
 (('HG00403', 'HG00404'), 1),
 (('HG00406', 'HG00407'), 1),
 (('HG00418', 'HG00419'), 1),
 (('HG00421', 'HG00422'), 1),
 (('HG00427', 'HG00428'), 1),
 (('HG00436', 'HG00437'), 1),
 (('HG00442', 'HG00443'), 1),
 (('HG00445', 'HG00446'), 1),
 (('HG00448', 'HG00449'), 1),
 (('HG00451', 'HG00452'), 1),
 (('HG00457', 'HG00458'), 1),
 (('HG00463', 'HG00464'), 1),
 (('HG00472', 'HG00473'), 1),
 (('HG00475', 'HG00476'), 1),
 (('HG00478', 'HG00479'), 1),
 (('HG00500', 'HG00501'), 1),
 (('HG00512', 'HG00513'), 1),
 (('HG00524', 'HG00525'), 1),
 (('HG00530', 'HG00531'), 1),
 (('HG00533', 'HG00534'), 1),
 (('HG00536', 'HG00537'), 1),
 (('HG00542', 'HG00543'), 1),
 (('HG01075', 'HG00551'), 1),
 (('HG00553', 'HG00554'), 1),
 (('HG00556', 'HG00557'), 1),
 (('HG00559', 'HG00560'), 1),
 (('HG00565', 'HG00566'),

In [22]:
kg_ped[kg_ped.FamilyID=='GBR001']

,FamilyID,SampleID,FatherID,MotherID,Sex,Population,Superpopulation
50,GBR001,HG00155,0,HG00144,1,GBR,EUR


In [6]:
# To copy/paste to pileup bash script.
kg_files = pd.read_csv('/scratch/groups/dpwall/personal/briannac/unmapped_reads/data/1kg_hg38_ids.txt', sep='\t', header=None)
kg_metadata = pd.read_csv('/scratch/groups/dpwall/personal/briannac/unmapped_reads/data/1kg_hg38_metadata.tsv', sep='\t')
kg_files.index = kg_files[9]
kg_metadata.index=kg_metadata['Sample name']

In [9]:
ids = list(set(kg_files.index).intersection(set(kg_metadata.index)))
kg_files = kg_files.loc[ids]

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
9,,,,,,,,,,,,,,,,,,,,,
HG00766,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR324/ERR324...,da0e327c506d7bf6f1cd7c7a52be1917,ERR3242129,ERP114329,30X whole genome sequencing coverage of the 25...,NYGC,ERA1783865,2019-03-22 00:00,SRS179212,HG00766,...,ILLUMINA,Illumina NovaSeq 6000,HG00766,ena-RUN-NYGC-22-03-2019-12:39:40:886-471,450,PAIRED,NaN,747102928,112065439200,high_cov
NA12400,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR323/ERR323...,c09b439ab04568b8bf67656f055d1b86,ERR3239478,ERP114329,30X whole genome sequencing coverage of the 25...,NYGC,ERA1783081,2019-03-22 00:00,SRS000629,NA12400,...,ILLUMINA,Illumina NovaSeq 6000,NA12400,ena-RUN-NYGC-22-03-2019-12:30:23:667-203,450,PAIRED,NaN,898974846,134846226900,high_cov
NA19700,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR323/ERR323...,26b9fdfd02c239bc043d4578bf52c98b,ERR3239913,ERP114329,30X whole genome sequencing coverage of the 25...,NYGC,ERA1783410,2019-03-22 00:00,SRS003668,NA19700,...,ILLUMINA,Illumina NovaSeq 6000,NA19700,ena-RUN-NYGC-22-03-2019-12:33:38:349-139,450,PAIRED,NaN,744943548,111741532200,high_cov
HG02178,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR324/ERR324...,d45cd3ebcd463dfc17e2e2e23899fd2e,ERR3242174,ERP114329,30X whole genome sequencing coverage of the 25...,NYGC,ERA1784271,2019-03-22 00:00,SRS179259,HG02178,...,ILLUMINA,Illumina NovaSeq 6000,HG02178,ena-RUN-NYGC-22-03-2019-12:43:59:740-16,450,PAIRED,NaN,737648030,110647204500,high_cov
HG00500,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR324/ERR324...,f5bfe1b900752ae38493c39a2ac1e3ad,ERR3241683,ERP114329,30X whole genome sequencing coverage of the 25...,NYGC,ERA1783865,2019-03-22 00:00,SRS008628,HG00500,...,ILLUMINA,Illumina NovaSeq 6000,HG00500,ena-RUN-NYGC-22-03-2019-12:39:40:869-25,450,PAIRED,NaN,806376892,120956533800,high_cov
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HG00632,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR324/ERR324...,f754219db5c3159967b4e426a80ee7b3,ERR3242520,ERP114329,30X whole genome sequencing coverage of the 25...,NYGC,ERA1784271,2019-03-22 00:00,SRS344120,HG00632,...,ILLUMINA,Illumina NovaSeq 6000,HG00632,ena-RUN-NYGC-22-03-2019-12:43:59:752-362,450,PAIRED,NaN,700382870,105057430500,high_cov
HG03199,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR324/ERR324...,209a463dcb4473ea7e49bcf520e9f2f4,ERR3242621,ERP114329,30X whole genome sequencing coverage of the 25...,NYGC,ERA1784271,2019-03-22 00:00,SRS350784,HG03199,...,ILLUMINA,Illumina NovaSeq 6000,HG03199,ena-RUN-NYGC-22-03-2019-12:43:59:755-463,450,PAIRED,NaN,738182354,110727353100,high_cov
HG03727,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR324/ERR324...,5c773cf81989486018da264987e20901,ERR3242704,ERP114329,30X whole genome sequencing coverage of the 25...,NYGC,ERA1784521,2019-03-22 00:00,SRS350903,HG03727,...,ILLUMINA,Illumina NovaSeq 6000,HG03727,ena-RUN-NYGC-22-03-2019-12:47:51:973-46,450,PAIRED,NaN,791610534,118741580100,high_cov


In [74]:
ids = list(set(kg_files.index).intersection(set(kg_metadata.index)))
kg_files = kg_files.loc[ids]
kg_metadata = kg_metadata.loc[ids]
"' '".join([i for i in kg_files[0].values[:20]])

"ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR324/ERR3242773/HG01092.final.cram' 'ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR324/ERR3242646/HG03793.final.cram' 'ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR324/ERR3242946/HG02879.final.cram' 'ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR324/ERR3242246/HG02292.final.cram' 'ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR323/ERR3239511/NA18613.final.cram' 'ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR324/ERR3241952/HG01617.final.cram' 'ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR324/ERR3242657/HG03829.final.cram' 'ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR323/ERR3239713/NA19318.final.cram' 'ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR324/ERR3242102/HG02049.final.cram' 'ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR324/ERR3243044/HG03767.final.cram' 'ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR323/ERR3239774/NA19461.final.cram' 'ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR324/ERR3240167/HG00180.final.cram' 'ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR323/ERR3239592/NA19060.final.cram' 'ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR323/ERR3239705/NA19308.fina

In [77]:
' '.join([i for i in kg_files[9].values[:20]])

'HG01092 HG03793 HG02879 HG02292 NA18613 HG01617 HG03829 NA19318 HG02049 HG03767 NA19461 HG00180 NA19060 NA19308 NA18975 HG03168 HG03291 NA19922 NA20755 NA18950'

In [79]:
Counter(kg_metadata.Sex[:20].values)

Counter({'female': 11, 'male': 9})

In [13]:
Counter(bam_mappings.loc[["07C65108","09C82723","07C70153","10C106894","10C106929","07C68567","07C63191","08C76306","10C115947","08C76307","07C65814","10C106936","10C117064","07C70151","09C87488","10C115948","10C117063","07C63192","07C70366","09C82721","07C65105","10C108230","07C70372","07C68568","07C65107","07C63425","09C82722","09C79835","10C106892","07C68566","07C65104","10C108279","10C115946","07C70365","10C106933","07C65103","07C65818","09C82724","10C106893"]
                ].sex_numeric.values)

Counter({'2.0': 30, '1.0': 9})

## Batch 1
sequencing_place = MTR-00027

In [10]:
'07C65108 09C82723 07C70153 10C106894 10C106929 07C68567 07C63191 08C76306 10C115947 08C76307 07C65814 10C106936 10C117064 07C70151 09C87488 10C115948 10C117063 07C63192 07C70366 09C82721 07C65105 10C108230 07C70372 07C68568 07C65107 07C63425 09C82722 09C79835 10C106892 07C68566 07C65104 10C108279 10C115946 07C70365 10C106933 07C65103 07C65818 09C82724 10C106893'.replace(' ', '","')

'07C65108","09C82723","07C70153","10C106894","10C106929","07C68567","07C63191","08C76306","10C115947","08C76307","07C65814","10C106936","10C117064","07C70151","09C87488","10C115948","10C117063","07C63192","07C70366","09C82721","07C65105","10C108230","07C70372","07C68568","07C65107","07C63425","09C82722","09C79835","10C106892","07C68566","07C65104","10C108279","10C115946","07C70365","10C106933","07C65103","07C65818","09C82724","10C106893'

In [55]:
print(len(bam_mappings[bam_mappings.sequencing_plate=='MTR-00027'].sample_id.values), ' people in batch')
' '.join(bam_mappings[bam_mappings.sequencing_plate=='MTR-00027'].sample_id.values)

95  people in batch


'03C14328 03C15416 03C15417 03C15418 03C15419 03C15790 03C15791 03C15962 03C15963 03C16097 03C16098 03C16794 03C16795 03C16796 03C16797 03C16798 10C102514 10C102515 10C102516 10C102517 10C103890 10C103891 10C103892 10C103893 10C104027 10C104028 10C104029 10C104030 10C104031 10C105023 10C110447 10C110448 10C110449 10C110450 10C110451 10C110758 10C110759 10C110760 10C110761 10C112856 10C112857 10C112858 10C112859 10C112904 11C120670 11C120671 11C120680 11C120681 11C120684 11C120729 11C120730 11C120731 11C120732 11C122939 11C122940 11C122941 11C122942 11C123571 11C123572 11C123573 11C123574 11C123749 11C123777 11C123778 11C125629 11C125630 11C125631 11C125632 11C125633 11C125634 11C125860 11C125861 11C125862 11C125875 11C125876 MH0131344 MH0131348 MH0131365 MH0135248 MH0137425 MH0137426 MH0137427 MH0137428 MH0137429 MH0138049 MH0138050 MH0138051 MH0138052 MH0138054 MH0138055 MH0143008 MH0143009 MH0143013 MH0143018 MH0143019'

## Family 1

In [3]:
fam = bam_mappings[bam_mappings['sample_id']=='MH0143019']['family'].values[0]
bam_mappings[bam_mappings['family']==fam]

,status,sample_id,participant_id,family,AGRE Blood,NIMH Blood,relationship,FatherID,MotherID,sex_numeric,...,number_of_indels,number_of_homozygous_snvs,number_of_heterozygous_snvs,het_to_hom_ratio,ti_tv_ratio,dbsnp_overlap,concordance_with_genotyping_chip,cryptic_relatedness_check,family_based_average_coverage,familial_relationship_check
4837,Passed_QC_analysis_ready,MH0143008,AU3859201,AU3859,HI11780,MH0143008,father,0,0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4838,Passed_QC_analysis_ready,MH0143009,AU3859202,AU3859,HI11781,MH0143009,mother,0,0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4839,Passed_QC_analysis_ready,MH0143013,AU3859303,AU3859,HI11784,MH0143013,sibling,AU3859201,AU3859202,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4840,Passed_QC_analysis_ready,MH0143018,AU3859301,AU3859,HI11782,MH0143018,sibling,AU3859201,AU3859202,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4841,Passed_QC_analysis_ready,MH0143019,AU3859302,AU3859,HI11783,MH0143019,sibling,AU3859201,AU3859202,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


family 1: MH0143008 MH0143009 MH0143013 MH0143018 MH0143019

**Note: These are WB**

## Family 2

In [4]:
bam_mappings[bam_mappings['family']=='AU1005']

,status,sample_id,participant_id,family,AGRE Blood,NIMH Blood,relationship,FatherID,MotherID,sex_numeric,...,number_of_indels,number_of_homozygous_snvs,number_of_heterozygous_snvs,het_to_hom_ratio,ti_tv_ratio,dbsnp_overlap,concordance_with_genotyping_chip,cryptic_relatedness_check,family_based_average_coverage,familial_relationship_check
0,Passed_QC_analysis_ready,02C10540,AU1005201,AU1005,HI2393,02C10540,father,0,0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Passed_QC_analysis_ready,02C10541,AU1005202,AU1005,HI2394,02C10541,mother,0,0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Passed_QC_analysis_ready,02C10542,AU1005302,AU1005,HI2395,02C10542,sibling,AU1005201,AU1005202,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Passed_QC_analysis_ready,02C10543,AU1005301,AU1005,HI2396,02C10543,sibling,AU1005201,AU1005202,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


family 2: 02C10540 02C10541 02C10542 02C10543

**Note: These are LCL**

## Family 3

In [50]:
seq_plate =bam_mappings[bam_mappings['family']=='AU3859']['sequencing_plate'].values[0]
bam_mappings[bam_mappings['sequencing_plate']==seq_plate].head(21)

,family,participant_id,sample_id,relationship,sex_numeric,status,batch_name,sequencing_plate,bio_seq_source
169,AU0162,AU016204,03C14328,sibling,2.0,Passed_QC_analysis_ready,WAL10680_B01,MTR-00027,LCL
230,AU0162,AU016201,03C15416,mother,2.0,Passed_QC_analysis_ready,WAL10680_B01,MTR-00027,LCL
231,AU0162,AU016202,03C15417,father,1.0,Passed_QC_analysis_ready,WAL10680_B01,MTR-00027,LCL
232,AU0162,AU016203,03C15418,sibling,1.0,Passed_QC_analysis_ready,WAL10680_B01,MTR-00027,LCL
233,AU0162,AU016205,03C15419,sibling,2.0,Sequencing_canceled,WAL10680_B01,MTR-00027,LCL
263,AU0236,AU023601,03C15790,mother,2.0,Passed_QC_analysis_ready,WAL10680_B01,MTR-00027,LCL
264,AU0236,AU023602,03C15791,father,1.0,Passed_QC_analysis_ready,WAL10680_B01,MTR-00027,LCL
302,AU0080,AU008001,03C15962,mother,2.0,Passed_QC_analysis_ready,WAL10680_B01,MTR-00027,LCL
303,AU0080,AU008002,03C15963,father,1.0,Passed_QC_analysis_ready,WAL10680_B01,MTR-00027,LCL
411,AU0080,AU008003,03C16097,sibling,2.0,Passed_QC_analysis_ready,WAL10680_B01,MTR-00027,LCL


Using family AU0041, another family from AU3859's/MH0143019's batch  ('MTR-00027')

sample IDS: 03C16794, 03C16795, 03C16796, 03C16797, 03C16798

**Note: These are LCL.**